In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
%matplotlib notebook
import pandas as pd

In [ ]:
pm_data = np.load('npy/pm_eOII_1deg.npy')
f_name = 'Source_cats/Source_cat_OII_1deg.npy'
mock = np.load(f_name, allow_pickle=True).item()
errors = np.load('npy/errors5Sigma.npy')[:,1]

In [ ]:
filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

In [ ]:
bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
# def stack_estimation(pm_flx, pm_err, nb_c, N_nb, w_central, nb_fwhm_Arr, ew0min):
#     '''
#     Returns the weighted average and std of N_nb Narrow Bands
#     arround the central one.
#     '''
#     nb_idx_Arr = np.array([*range(nb_c-N_nb, nb_c+N_nb+1)])
    
#     flx = pm_flx[nb_idx_Arr]
#     err = pm_err[nb_idx_Arr]
    
#     # Remove NB compatible with emission lines
#     ref_bb = -3
#     ew0min = 50
    
#     z = 1215.67/w_central[nb_c] - 1
#     ew = (1 + z) * ew0min

#     bbnb = flx - pm_flx[ref_bb]
#     bbnb_err = (err**2 + pm_err[ref_bb]**2)**0.5
#     outliers = bbnb.T > 3*bbnb_err.T + ew*pm_flx[ref_bb].reshape(-1, 1)\
#                                         /np.array(nb_fwhm_Arr)[nb_idx_Arr]
#     out = np.where(outliers)
#     out_symmetric = (out[0], N_nb - (out[1]-N_nb))
#     err[out[1], out[0]] = 999.
#     err[out_symmetric[1], out_symmetric[0]] = 999.
#     err[N_nb] = 9999.

    
#     avg = np.average(flx, axis=0, weights=1./err)
#     std = np.average((flx - avg)**2, axis=0, weights=1./err)**0.5
    
#     return avg, std

In [ ]:
# The model function
def model_f(x, m, b):
    return m*x + b 

In [ ]:
nb_c = 10
N_nb_fit = 10
N_nb_stack = 5
ew0min = 50
nb_fwhm_Arr = [nb_fwhm(tcurves, idx, True) for idx in np.arange(len(filters_tags))]
err = (np.ones(pm_data.shape).T * errors).T
cont_stack, cont_err_stack = stack_estimation(
    pm_data, err, nb_c, N_nb_stack, w_central, nb_fwhm_Arr, ew0min
)
_, cont_fit, cont_err_fit = nbex_cont_estimate(
    model_f, pm_data.T, err.T, nb_c, w_central, N_nb_fit, ew0min, nb_fwhm_Arr[10]
)

In [ ]:
plt.close('all')
for mock_n in range(15):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots()
    ax.plot( mock['w_Arr'], mock['SEDs'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
#     ax.scatter(w_central[nb_c], pm[nb_c], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c]+10, cont_stack[mock_n], yerr=cont_err_stack[mock_n],
                c='violet', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    cont_fit_value = cont_fit[mock_n, 1] + cont_fit[mock_n, 0]*w_central[nb_c]
    ax.plot(np.linspace(4000,6000,1000),
            cont_fit[mock_n, 1] + cont_fit[mock_n, 0]*np.linspace(4000,6000,1000),
            c='saddlebrown', linestyle='dashed')
    ax.errorbar(w_central[nb_c], cont_fit_value, yerr=cont_err_fit[mock_n],
                c='saddlebrown', marker='*', markersize=9,
                capsize=4, label='Linear fit', elinewidth=2, capthick=2)
    
    plt.legend()

    plt.show()

In [ ]:
cont_fit_value = cont_fit[:,1] + cont_fit[:,0]*w_central[nb_c]

real_cont_value = np.zeros(pm_data.shape[1])
for i in range(pm_data.shape[1]):
    real_cont_value[i] = np.interp(w_central[nb_c], mock['w_Arr'], mock['SEDs_no_line'][i])

In [ ]:
diff_df = pd.DataFrame(
    {
        'gSDSS_diff':    (pm_data[-3, :] - real_cont_value) / real_cont_value,
        'Stack_diff':    (cont_stack - real_cont_value) / real_cont_value,
        'Fit_diff'  :    (cont_fit_value - real_cont_value) / real_cont_value,
        'gSDSS_diff_abs':    np.abs((pm_data[-3, :] - real_cont_value) / real_cont_value),
        'Stack_diff_abs':    np.abs((cont_stack - real_cont_value) / real_cont_value),
        'Fit_diff_abs'  :    np.abs((cont_fit_value - real_cont_value) / real_cont_value)
    }
)

In [ ]:
diff_df.head(20)[['gSDSS_diff', 'Stack_diff', 'Fit_diff']]

In [ ]:
diff_df.describe().loc[['mean', '50%', 'std']]